In [1]:
import tensorflow as tf
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, accuracy_score
from keras.datasets import mnist
import time
from tensorflow.keras import layers
from tensorflow.keras import Sequential

In [2]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1]*X_train.shape[2]))/255
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1]*X_test.shape[2]))/255


In [3]:
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(60000, 784) (60000,) (10000, 784) (10000,)


In [4]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [5]:
param = {
    'eta': 0.3, 
    'max_depth': 5,  
    'objective': 'multi:softprob',  
    'num_class': 10} 

steps = 20  # The number of training iterations

start = time.time()
model_xgb = xgb.train(param, D_train, steps)
end = time.time()
print('Training Time:', end-start)

[14:05:08] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training Time: 16.75418448448181


In [6]:
preds = model_xgb.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9502932542595254
Recall = 0.9497896352905887
Accuracy = 0.9504


In [7]:
y_train = np.zeros((Y_train.shape[0], Y_train.max()+1), dtype=np.float32)
y_train[np.arange(Y_train.shape[0]), Y_train] = 1
y_test = np.zeros((Y_test.shape[0], Y_test.max()+1), dtype=np.float32)
y_test[np.arange(Y_test.shape[0]), Y_test] = 1

In [8]:
# get the model
def get_model1(n_inputs, n_outputs):
    model = Sequential()
    model.add(layers.Dense(350, input_dim=n_inputs, kernel_initializer='he_uniform', activation='sigmoid'))
    model.add(layers.Dense(120, input_dim=n_inputs, kernel_initializer='he_uniform', activation='sigmoid'))
    model.add(layers.Dense(n_outputs,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return model

def get_model2(n_inputs, n_outputs):
    model = Sequential()
    model.add(layers.Dense(500, input_dim=n_inputs, kernel_initializer='he_uniform', activation='sigmoid'))
    model.add(layers.Dense(250, input_dim=n_inputs, kernel_initializer='he_uniform', activation='sigmoid'))
    model.add(layers.Dense(100, input_dim=n_inputs, kernel_initializer='he_uniform', activation='sigmoid'))
    model.add(layers.Dense(n_outputs,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return model

model_dnn1 = get_model1(28*28,10)
model_dnn1.summary()

model_dnn2 = get_model2(28*28,10)
model_dnn2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 350)               274750    
_________________________________________________________________
dense_1 (Dense)              (None, 120)               42120     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1210      
Total params: 318,080
Trainable params: 318,080
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 500)               392500    
_________________________________________________________________
dense_4 (Dense)              (None, 250)               125250    
____________________________

In [9]:
batch_size = 64
epochs = 50

start = time.time()
history_dnn1 = model_dnn1.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,validation_data=(X_test, y_test))
end = time.time()
print('Training Time:', end-start)

Epoch 1/50
938/938 [==============================] - 2s 1ms/step - loss: 0.4430 - categorical_accuracy: 0.8821 - val_loss: 0.2170 - val_categorical_accuracy: 0.9347
Epoch 2/50
938/938 [==============================] - 1s 1ms/step - loss: 0.1853 - categorical_accuracy: 0.9446 - val_loss: 0.1475 - val_categorical_accuracy: 0.9554
Epoch 3/50
938/938 [==============================] - 1s 1ms/step - loss: 0.1301 - categorical_accuracy: 0.9615 - val_loss: 0.1137 - val_categorical_accuracy: 0.9645
Epoch 4/50
938/938 [==============================] - 1s 1ms/step - loss: 0.0960 - categorical_accuracy: 0.9713 - val_loss: 0.0937 - val_categorical_accuracy: 0.9712
Epoch 5/50
938/938 [==============================] - 1s 1ms/step - loss: 0.0732 - categorical_accuracy: 0.9783 - val_loss: 0.0856 - val_categorical_accuracy: 0.9741
Epoch 6/50
938/938 [==============================] - 1s 1ms/step - loss: 0.0573 - categorical_accuracy: 0.9828 - val_loss: 0.0877 - val_categorical_accuracy: 0.9723
Epoc

Epoch 50/50
938/938 [==============================] - 1s 1ms/step - loss: 1.4196e-05 - categorical_accuracy: 1.0000 - val_loss: 0.0967 - val_categorical_accuracy: 0.9835
Training Time: 67.5883264541626


In [10]:
start = time.time()
history_dnn2 = model_dnn2.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,validation_data=(X_test, y_test))
end = time.time()
print('Training Time:', end-start)

Epoch 1/50
938/938 [==============================] - 2s 2ms/step - loss: 0.4613 - categorical_accuracy: 0.8676 - val_loss: 0.2230 - val_categorical_accuracy: 0.9304
Epoch 2/50
938/938 [==============================] - 1s 2ms/step - loss: 0.1697 - categorical_accuracy: 0.9494 - val_loss: 0.1466 - val_categorical_accuracy: 0.9576
Epoch 3/50
938/938 [==============================] - 2s 2ms/step - loss: 0.1147 - categorical_accuracy: 0.9656 - val_loss: 0.1091 - val_categorical_accuracy: 0.9653
Epoch 4/50
938/938 [==============================] - 2s 2ms/step - loss: 0.0842 - categorical_accuracy: 0.9750 - val_loss: 0.1023 - val_categorical_accuracy: 0.9680
Epoch 5/50
938/938 [==============================] - 2s 2ms/step - loss: 0.0643 - categorical_accuracy: 0.9803 - val_loss: 0.0903 - val_categorical_accuracy: 0.9720
Epoch 6/50
938/938 [==============================] - 1s 1ms/step - loss: 0.0496 - categorical_accuracy: 0.9847 - val_loss: 0.0730 - val_categorical_accuracy: 0.9765
Epoc

Epoch 50/50
938/938 [==============================] - 1s 1ms/step - loss: 0.0014 - categorical_accuracy: 0.9995 - val_loss: 0.1044 - val_categorical_accuracy: 0.9837
Training Time: 70.72793960571289
